<a href="https://colab.research.google.com/github/hogo56/BertQA/blob/master/ColabGettingStarted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## --Setup--

### Set Tensorflow version
TPU support is scheduled for TF 2.1+ so currently set to GPU if using TF 2.x until google puts 2.1 in colab.

In [0]:
%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)

### Load Libraries

In [0]:
import sys
import os
import numpy as np
import pandas as pd
import json
import absl
from zipfile import ZipFile

In [0]:
# import tf2_0_baseline_w_bert_translated_to_tf2baseline # Oliviera's script
# import bert_modeling as modeling
# import bert_optimization as optimization
# import bert_tokenization as tokenization

## --Data Transfer--

In [0]:
!mkdir -p /content/data

### Upload from local machine


In [0]:
from google.colab import files
uploaded = files.upload()
print(uploaded)

# model.save('trained_model.h5')
# files.download('trained_model.h5')

### Through Google Drive
<Details>To connect to Drive you will have to grant Google Drive File Stream access your Google Account then copy the auth token back here.<br>
**Warning:** you will be granting access by your colab to your entire gdrive<br>
To Revoke colab access: https://myaccount.google.com/permissions<br>
<br>
Note: file access to Drive may be a performance bottle neck, maybe copy files to colab or try a memory mapped file formats like HDF5 (aka H5) or LMDB.</Details>

In [0]:
## File link to Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)   # true to reread drive
projDir = '/content/gdrive//My Drive/BartQA/'

# drive.flush_and_unmount()  # changes made in session should be visible in Drive

## Through Kaggle API
<Details> You will have to go to www.kaggle.com -> My Account -> Create New API Token to get a kaggle.json file. Then upload it here</Details>

In [22]:
## Authenticate to Kaggle

from google.colab import files
print('Upload kaggle.json')
!rm /content/kaggle.json
uploaded = files.upload()
!chmod 600 kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
!ls -l /content/kaggle.json

import kaggle


Upload kaggle.json
rm: cannot remove '/content/kaggle.json': No such file or directory


Saving kaggle.json to kaggle.json
total 24
drwxr-xr-x 2 root root 4096 Dec 11 11:46  data
drwx------ 3 root root 4096 Dec 11 10:23  gdrive
-rw-r--r-- 1 root root   66 Dec 11 12:17 'kaggle (1).json'
-rw-r--r-- 1 root root   66 Dec 11 12:18 'kaggle (2).json'
-rw------- 1 root root   66 Dec 11 12:24  kaggle.json
drwxr-xr-x 1 root root 4096 Dec  6 16:53  sample_data


In [0]:
!kaggle datasets list
# !kaggle competitions list
# !kaggle competitions download -c tensorflow2-question-answering -p /content/data  # 5GB

In [0]:
!ls -lh /content/data

### From Web
<Details>You can also use !wget</Details>

In [0]:
from urllib.request import urlretrieve

def download(url, file):
    if not os.path.isfile(file):
        print("Downloading file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download('Url of the file','Name of the file to be saved')
print("All the files are downloaded")
#If the downloaded file is a zip file than you can use below function to unzip it.
def uncompress_features_labels(file):
    if(os.path.isdir('data')):
        print('Data extracted')
    else:
        with ZipFile(dir) as zipf:
            zipf.extractall('data')

## --Explore Environment--


In [0]:
if True:
    # !cat /proc/cpuinfo            # You can run bash commands
    # !cat /proc/meminfo
    # from tensorflow.python.client import device_lib
    # print()
    # print(device_lib.list_local_devices())
    # print()
    # print("Dir of /:", *(os.listdir('/')), sep='\n')
    # print("\ncwd: ", os.getcwd())                       # will be /content
    for dirname, _, filenames in os.walk(os.getcwd()):
        for filename in filenames:
            print(os.path.join(dirname, filename))

In [0]:
!nvidia-smi
print()
!/usr/local/cuda/bin/nvcc --version

## --ToDo--

Figure out how to save trainig, data, test data, backups, etc to drive during model run<P>
Setup SSH

In [0]:
# https://developer.nvidia.com/cudnn

# Extracts the cuDNN files from Drive folder directly to the VM CUDA folders
!tar -xzvf gdrive/My\ Drive/BartQA/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
